In [4]:
import pandas as pd
data_test = pd.read_csv('test.csv')
data_train = pd.read_csv('train.csv')


In [5]:
x = data_test[['Gender', 'Class', 'Flight Distance', 'Time convenient', 'Ease of booking', 'Onboard service', 'Inflight service']]
y = data_test[data_test.columns[-1]]


In [6]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# le = LabelEncoder()

categorical_columns = ['Gender', 'Customer Type', 'Type of Travel', 'Class']
ohencod = OneHotEncoder()
# melatih model dan mentransformasi data input
encoded_data = ohencod.fit_transform(data_train[categorical_columns])

# mengubah menjadi DataFrame
encoded_df = pd.DataFrame(
    encoded_data.toarray(), 
    columns=['Female', 'Male', 'disLoyal', 'Loyal', 'Business', 
             'Personal', 'Business_Class', 'Eco_Class', 'Eco_Plus_Class'])

# Menggabungkan kembali data yang sudah diencoding
features = pd.concat([encoded_df, data_train.drop(categorical_columns, axis=1)], axis=1)


x = features.drop(['Satisfaction'], axis=1)
y = features['Satisfaction']
x.fillna(x.mean(), inplace=True)

# Mengaplikasikan SelectKBest untuk mencari 7 fitur terbaik
best_features = SelectKBest(score_func=f_classif, k=7)
fit = best_features.fit(x, y)
# Mendapatkan 7 fitur terbaik
X_new = fit.transform(x)

# Mendapatkan array boolean dari fitur yang didapat
mask = fit.get_support()

# Mendapatkan nama-nama fitur 
selected_features = []
for bool, feature in zip(mask, data_train.columns):
    if bool:
        selected_features.append(feature)

print(selected_features)


['Class', 'Flight Distance', 'Inflight wifi service', 'Time convenient', 'Baggage handling', 'Checkin service', 'Inflight service']


In [7]:
feature_train = data_train[selected_features]
target_train = data_train['Satisfaction']

feature_test = data_test[selected_features]
target_test = data_test['Satisfaction']

print("\nFeature matrix : \n", x)
print("\nResponse vector : \n", y)


Feature matrix : 
         Female  Male  disLoyal  Loyal  Business  Personal  Business_Class  \
0          0.0   1.0       1.0    0.0       0.0       1.0             0.0   
1          0.0   1.0       0.0    1.0       1.0       0.0             1.0   
2          1.0   0.0       1.0    0.0       1.0       0.0             1.0   
3          1.0   0.0       1.0    0.0       1.0       0.0             1.0   
4          0.0   1.0       1.0    0.0       1.0       0.0             1.0   
...        ...   ...       ...    ...       ...       ...             ...   
103899     1.0   0.0       0.0    1.0       1.0       0.0             0.0   
103900     0.0   1.0       1.0    0.0       1.0       0.0             1.0   
103901     0.0   1.0       0.0    1.0       1.0       0.0             1.0   
103902     1.0   0.0       0.0    1.0       1.0       0.0             0.0   
103903     0.0   1.0       1.0    0.0       1.0       0.0             1.0   

        Eco_Class  Eco_Plus_Class  Age  ...  Seat comfo

<h1>Proses encoding menggunakan LabelEncoder

In [8]:
# mengubah menjadi DataFrame
encoded_data_train = pd.DataFrame(
    ohencod.fit_transform(feature_train[['Class']]).toarray(), 
    columns=['Business_Class', 'Eco_Class', 'Eco_Plus_Class'])

encoded_data_test = pd.DataFrame(
    ohencod.fit_transform(feature_test[['Class']]).toarray(), 
    columns=['Business_Class', 'Eco_Class', 'Eco_Plus_Class'])

# Menggabungkan kembali data yang sudah diencoding
data_train_final = pd.concat([encoded_data_train, feature_train.drop(['Class'], axis=1)], axis=1)
data_train_final.fillna(data_train_final.mean(), inplace=True)

data_test_final = pd.concat([encoded_data_test, feature_test.drop(['Class'], axis=1)], axis=1)
data_test_final.fillna(data_test_final.mean(), inplace=True)
print("\nFeature matrix : \n", data_train_final)


Feature matrix : 
         Business_Class  Eco_Class  Eco_Plus_Class  Flight Distance  \
0                  0.0        0.0             1.0              460   
1                  1.0        0.0             0.0              235   
2                  1.0        0.0             0.0             1142   
3                  1.0        0.0             0.0              562   
4                  1.0        0.0             0.0              214   
...                ...        ...             ...              ...   
103899             0.0        1.0             0.0              192   
103900             1.0        0.0             0.0             2347   
103901             1.0        0.0             0.0             1995   
103902             0.0        1.0             0.0             1000   
103903             1.0        0.0             0.0             1723   

        Inflight wifi service  Time convenient  Baggage handling  \
0                           3                4                 4   
1  

In [9]:
from sklearn.neighbors import KNeighborsClassifier

x_test = data_test_final
y_test = target_test
x_train = data_train_final
y_train = target_train

k = [3, 5, 7, 9, 11]

for i in k:
    knn = KNeighborsClassifier(n_neighbors=i)
    # melatih model
    knn.fit(x_train, y_train)
    # menghitung akurasi prediksi model pada data uji
    accuracy = knn.score(x_test, y_test)
    print(f'Akurasi KNN dengan k:{i} = {accuracy}')

Akurasi KNN dengan k:3 = 0.8129812134277795
Akurasi KNN dengan k:5 = 0.8163304588851247
Akurasi KNN dengan k:7 = 0.8152525408068987
Akurasi KNN dengan k:9 = 0.8122112719433323
Akurasi KNN dengan k:11 = 0.8116723129042193
